## Gradient Descent on a Simple Parabola

In this notebook, we are going to learn how to perform gradient descent on a very simple function. Let's look at this parabola: 

$ f(x) = (x-3)^2 + 4 $

We want to find the $ x $ value that will produce the minimum $ y $ value. To visualize this, let's use [Lets-Plot](https://blog.jetbrains.com/kotlin/2020/12/lets-plot-in-kotlin/). It comes with the Kotlin Jupyter kernel which can be installed with the command below, which is also required to run this notebook. 

```shell
conda install kotlin-jupyter-kernel -c jetbrains
```

Then lets enable Lets-Plot in this notebook. 

In [1]:
%use lets-plot

Now we are good to go. Plot the parabola using this Kotlin code below:

In [2]:
// simple parabola function
fun f(x: Double) = (x - 3).pow(2) + 4

// define x-range
val xStart = 0.0
val xEnd = 6.0

// generate x-values to fill the x-range
val xRange = generateSequence(xStart) { it + .01 }.takeWhile { it < xEnd }.toList()

//pass each x-value to the parabola function
val yParabola = xRange.map(::f)

// declare scatterplot 
val p = letsPlot { x = xRange; y = yParabola } + ggsize(600, 500)
p + geomPoint(size = 1) + geomPoint(x = 3.0, y = 4.0, size = 5, color="red")

Note that we can visibly see the minimum of this parabola is at $ (x = 3, y = 4 )$. Let's use this simple toy problem to understand differentiable programming and gradient descent to arrive at this minimum. This will empower you to understand more complex examples later ranging from physics and logistic regression models to neural networks.

Let's bring in the DiffKT API. We are going to use it to take the derivative of this function, enabling us to find the slope anywhere along this parabola. 

In [3]:
@file:DependsOn("../kotlin/api/build/libs/api.jar")

Let's determine the slope of our function at $ x = 2 $. We will declare that as a `FloatScalar` and modify our function `f` to accept a `DScalar` input variable `x`, declaring our parabola using the differentiable API. Finally, we call `primalAndForwardDerivative` to retrieve both the $ y $ value and the `y_slope` which is $ y' $. In this case, the slope $ y' $ ` is $ - 2.0 $. 

In [4]:
import org.diffkt.*


// simple quadratic function
val x = FloatScalar(2F)
fun f(x : DScalar) = (x - 3F).pow(2F) + 4F

val (y, ySlope) = primalAndForwardDerivative(x, ::f)

// evaluation of quadratic function
println("x = $x") // 2.0F 
println("y = $y") // 5.0F 
println("ySlope = $ySlope") // -2.0F 

x = 2.0
y = 5.0
ySlope = -2.0


There are two types of differentiation: forward and reverse. In this case we used forward hence the call to `primalAndForwardDerivative()`. Forward takes the derivative with the respect to the input, while reverse takes the derivative with respect to the output. Forward differentiation is advantageous when there are few inputs and many outputs, while reverse differentiation works best when there are few inputs and many outputs. Since we are only dealing with one input and one output with this simple function `f()`, we are going to get the same answer either way. If we were to call `primalAndReverseDerivative()` with the same arguments, we would still get a slope of $ -2.0 $. 

If you want to visualize this, we can think of the slope as a tangent line that just touches the parabola at $ x = 2 $. Let's modify our graph to show this tangent line visualizing the slope. The tangent line can be calculated algebraically, as we already have the $ x $, $ y $, and $ y' $ values. We just isolate the intercept $ b $ in our linear function. 

$ y = y'x + b $

$ y - y'x = b $   

In [5]:
val xStart = 0.0
val xEnd = 6.0

// generate parabola
val xRange = generateSequence(xStart) { it + .01 }.takeWhile { it < xEnd }.toList()
val yParabola = xRange.map { (f(FloatScalar(it.toFloat())) as FloatScalar).value }

val p = letsPlot { x = xRange; y = yParabola } + ggsize(600, 500)

p + geomPoint(size = 1) + geomABLine(
    slope = (ySlope as FloatScalar).value.toDouble(), 
    intercept = ((y - ySlope * x) as FloatScalar).value,  
    color="orange", 
    size = 1
) 


Eyeing the graph above, we can clearly see the tangent line at $ x = 2 $ and the resulting slope $ y'$ is $ -2 $. Now let's apply gradient descent to find the $ x $ value that produces the minimum $ y $ value, where the slope  $ y' $ is going to be 0 resulting in a flat line. 

Imagine that parabola is a mountain range. You find yourself on one of those slopes and you want to get to the bottom. However it is dark and you only have a flashlight. What you can do is look around to see where the slope is descending, and logically that will take you to the lowest point if you follow it. To make the process faster, you can take bigger steps if the slope is steep. A steep slope indicates you are not near the minimum yet. But if the slope starts to get shallow then we are approaching a flat slope, where we expect the minimum. In this case we want to take smaller steps so we do not step over the minimum. Finally, when the slope is 0 we are at the minimum!

This is essentially what gradient descent is. We move along our landscape following the slope towards the minimum. We take bigger steps for bigger slopes, and smaller steps for smaller slopes. This proportional relationship between slope and the step size can be scaled with a **learning rate**, which makes our steps a fraction of the slope. 

We also need enough steps, called **epochs** or **iterations**, in a `for` loop to bring us to our minimum. However, having too few iterations will end our search prematurely, but having too many will result in a diminishing return when we arrive at the minimum. On top of that, making our learning rate too large will cause us to keep stepping over the minimum like a giant, but having it too small makes our steps tiny like an insect's, resulting in an ineffecient number of iterations. 

Let's put all this together. For 100,000 iterations and a learning rate of .001, let's walk our `x` value to the minimum of the parabola. On each iteration, we will subtract a fraction of the slope (specified by learning rate `L`) to keep nudging `x` towards the minimum. 

In [6]:
import kotlin.random.Random 

// The learning rate
val L = FloatScalar(.001F)

// The number of iterations to perform gradient descent
val iterations = 100_000

// Start x at a random location in the parabola
var x: DScalar = FloatScalar(Random.nextFloat() * 6F) 

// Perform gradient descent 
for (i in 0..iterations) { 
    
    // get gradient/slope
    val (y, ySlope) = primalAndForwardDerivative(x, ::f)
    
    // update x by subtracting the (learning rate) * (slope)
    x = x - (L * ySlope)
}

// Calculate final y and y' 
val (y, ySlope) = primalAndForwardDerivative(x, ::f)

println("x = $x, y = $y, y' = $ySlope")

x = 2.9999404, y = 4.0, y' = -1.1920929E-4


Wow, sure enough you get (or extremely close to) $ x = 3 $ and $ y = 4.0 $, with a $ y' $ extremely close to 0. Why did we get a close approximation rather than an exact answer? Well gradient descent is an approximation technique, and you can improve that approximation with more iterations and smaller learning rate, but with diminishing returns. 

For good measure, let's plot our parabola again with our final gradient/slope. Sure enough our tangent line ends up on the bottom of the parabola and we found our minimum at $ x = 3 $. 


In [7]:
p + geomPoint(size = 1) + geomABLine(
    slope = (ySlope as FloatScalar).value.toDouble(), 
    intercept = ((y - ySlope * x) as FloatScalar).value,  
    color="orange", 
    size = 1
) 

While there are better ways to solve a simple problem like this (e.g. algebraically), gradient descent comes in handy for machine learning, physics, and other demanding problems that can only be approximated. With this toy example, you have a grasp of all the moving parts in gradient descent. 

Of course, this problem has only one local minimum, automatically making it the global minimum. A **local minimum** is a valley where a minimum resides within part of the function, but the **global minimum** is the lowest minimum across all the valleys across all possible values in a function. Gradient descent works well for **convex problems** which have only one local minimum or maximum. Convex problems include linear regression, logistic regression, and linear programming. 

However, when many local minima exists this creates opportunities for the gradient descent algorithm to get stuck in a single local minimum. Take a look at this function $ f(x) = cos(3 \pi x) / x $ and note how it has several local minimums. This will be covered in a separate notebook. 

In [8]:
val xRange = generateSequence(0.1) { it + .0001 }.takeWhile { it < 1.1 }.toList()
val yCurve = xRange.map { cos(3*Math.PI*it) / it  }

val p = letsPlot { x = xRange; y = yCurve } 

p + geomPoint(size = 1) 